In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pylab as plt
import seaborn as sns
import gc

import lightgbm as lgb
import ngboost as ngb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

from scipy import stats

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth',100)
pd.set_option('display.width',100)
pd.set_option('display.max_info_rows',100)

In [2]:
gc.collect()

31

In [3]:
# train = pd.read_csv('./jet_simple_data/simple_train_R04_jet.csv')
# test = pd.read_csv('./jet_simple_data/simple_test_R04_jet.csv')
data = pd.read_feather('E:/jet_complex_data/data_new.feather')
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548097 entries, 0 to 548096
Data columns (total 798 columns):
event_id                                            548097 non-null object
label                                               371377 non-null float32
count                                               548097 non-null int8
event_id_number_of_particles_in_this_jet_mean       548097 non-null float32
event_id_number_of_particles_in_this_jet_max        548097 non-null int8
event_id_number_of_particles_in_this_jet_min        548097 non-null int8
event_id_number_of_particles_in_this_jet_sum        548097 non-null int16
event_id_number_of_particles_in_this_jet_std        548097 non-null float32
event_id_jet_px_mean                                548097 non-null float32
event_id_jet_px_max                                 548097 non-null float32
event_id_jet_px_min                                 548097 non-null float32
event_id_jet_px_sum                                 548097 non-n

In [4]:
data['count'].value_counts()

2     227084
3     164229
4      88351
5      40187
6      16698
7       6184
1       2338
8       2078
9        695
10       188
11        46
12        17
14         1
13         1
Name: count, dtype: int64

In [5]:
data['isTest'] = data['label'].isna()
train = data.loc[data['isTest']==False]
test = data.loc[data['isTest']==True]

In [6]:
label_map = dict(zip(np.sort(train['label'].unique()), range(len(train['label'].unique()))))
label_map_reverse = dict(zip(range(len(train['label'].unique())), np.sort(train['label'].unique())))

In [7]:
# train['label'] = train['label'].map(label_map)
# imp_array = np.load('E:/jet_complex_data/imp.npy', allow_pickle=True)
# imp_array
# len(imp_array)

array(['event_id_jet_px_sum', 'event_id_jet_px_mean',
       'event_id_jet_id_particle_py_std_min', ...,
       'event_id_particle_category_13_min',
       'event_id_jet_id_particle_mass/pt_min_min',
       'event_id_jet_id_particle_mass_min_min'], dtype=object)

1540

In [8]:
drop_feat = ['event_id']
# for x in imp_array[:1530:-1]:
#     drop_feat.append(x)
    
    
# for col in data.columns.to_list():
# #     if 'kurt' in col or 'q1'in col or 'q3'in col:
#     if 'q3'in col:
#         print(col)
#         drop_feat.append(col)

In [9]:
used_feat = [f for f in test.columns if f not in ['label', 'jet_id', 'isTest'] + list(drop_feat)]
len(used_feat)
used_feat

796

['count',
 'event_id_number_of_particles_in_this_jet_mean',
 'event_id_number_of_particles_in_this_jet_max',
 'event_id_number_of_particles_in_this_jet_min',
 'event_id_number_of_particles_in_this_jet_sum',
 'event_id_number_of_particles_in_this_jet_std',
 'event_id_jet_px_mean',
 'event_id_jet_px_max',
 'event_id_jet_px_min',
 'event_id_jet_px_sum',
 'event_id_jet_px_std',
 'event_id_jet_py_mean',
 'event_id_jet_py_max',
 'event_id_jet_py_min',
 'event_id_jet_py_sum',
 'event_id_jet_py_std',
 'event_id_jet_pz_mean',
 'event_id_jet_pz_max',
 'event_id_jet_pz_min',
 'event_id_jet_pz_sum',
 'event_id_jet_pz_std',
 'event_id_jet_energy_mean',
 'event_id_jet_energy_max',
 'event_id_jet_energy_min',
 'event_id_jet_energy_sum',
 'event_id_jet_energy_std',
 'event_id_jet_mass_mean',
 'event_id_jet_mass_max',
 'event_id_jet_mass_min',
 'event_id_jet_mass_sum',
 'event_id_jet_mass_std',
 'event_id_jet_energy/num_mean',
 'event_id_jet_energy/num_max',
 'event_id_jet_energy/num_min',
 'event_id_j

In [10]:
train_x = train[used_feat].reset_index(drop=True)
train_y = train['label'].reset_index(drop=True)
test_x = test[used_feat].reset_index(drop=True)

In [11]:
def multi_auc(y_true, y_score):
    score =  roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(np.argmax(y_score.reshape((-1,4)), axis=1)))
    return 'multi_auc', score, True

def multi_auc_oof(y_true, y_score):
    score =  roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(np.argmax(y_score, axis=1)))
    return score

In [12]:
preds = np.zeros((test_x.shape[0], 4))
oof_lgb = np.zeros((train_x.shape[0], 4))
scores = []

imp = pd.DataFrame()
imp['feat'] = used_feat

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=527)
for index, (tr_idx, va_idx) in enumerate(kfold.split(train_x, train_y)):
    print('*'*120)
    print('Fold %d' % (index+1))
    print('*'*120)
    X_train, y_train, X_valid, y_valid = train_x.iloc[tr_idx], train_y.iloc[tr_idx], train_x.iloc[va_idx], train_y.iloc[va_idx]
    eval_set = [(X_valid, y_valid)]
    model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=2**6-1, max_depth=6, reg_alpha=0, reg_lambda=1.9, 
                                    max_bin=64, n_estimators=10000, objective='multiclass', num_class=4, 
                                    bagging_fraction=0.8, bagging_freq=5, min_child_samples=80, 
                                    feature_fraction=0.8, learning_rate=0.05, random_state=527, n_jobs=4)
#     model = ngb.NGBClassifier()
    
    model.fit(X_train, y_train, eval_set=eval_set,verbose=300, early_stopping_rounds=500,
#                   eval_metric=multi_auc, 
#                   categorical_feature=catlist
                 )
    
    imp['score%d' % (index+1)] = model.feature_importances_
    
    score = model.best_score_['valid_0']['multi_logloss']
    scores.append(score)
    print('fold %d round %d : score: %.6f | mean score %.6f' % (index+1, model.best_iteration_, score,np.mean(scores)))
    proba =  model.predict_proba(X_valid)
    print('auc: ',multi_auc_oof(y_valid, proba))
    print('\n\n')
    oof_lgb[va_idx] = proba
    preds += model.predict_proba(test_x)

************************************************************************************************************************
Fold 1
************************************************************************************************************************
Training until validation scores don't improve for 500 rounds
[300]	valid_0's multi_logloss: 0.858356
[600]	valid_0's multi_logloss: 0.837407
[900]	valid_0's multi_logloss: 0.82846
[1200]	valid_0's multi_logloss: 0.824028
[1500]	valid_0's multi_logloss: 0.821547
[1800]	valid_0's multi_logloss: 0.819763
[2100]	valid_0's multi_logloss: 0.819179
[2400]	valid_0's multi_logloss: 0.818776
[2700]	valid_0's multi_logloss: 0.818445
[3000]	valid_0's multi_logloss: 0.818393
[3300]	valid_0's multi_logloss: 0.818794
Early stopping, best iteration is:
[2933]	valid_0's multi_logloss: 0.818342


LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.05, max_bin=64,
               max_depth=6, min_child_samples=80, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=10000, n_jobs=4, num_class=4,
               num_leaves=63, objective='multiclass', random_state=527,
               reg_alpha=0, reg_lambda=1.9, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

fold 1 round 2933 : score: 0.818342 | mean score 0.818342
auc:  0.7723285971455489



************************************************************************************************************************
Fold 2
************************************************************************************************************************
Training until validation scores don't improve for 500 rounds
[300]	valid_0's multi_logloss: 0.858002
[600]	valid_0's multi_logloss: 0.836573
[900]	valid_0's multi_logloss: 0.827798
[1200]	valid_0's multi_logloss: 0.82388
[1500]	valid_0's multi_logloss: 0.820907
[1800]	valid_0's multi_logloss: 0.819043
[2100]	valid_0's multi_logloss: 0.818185
[2400]	valid_0's multi_logloss: 0.817776
[2700]	valid_0's multi_logloss: 0.817598
[3000]	valid_0's multi_logloss: 0.817802
Early stopping, best iteration is:
[2683]	valid_0's multi_logloss: 0.817544


LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.05, max_bin=64,
               max_depth=6, min_child_samples=80, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=10000, n_jobs=4, num_class=4,
               num_leaves=63, objective='multiclass', random_state=527,
               reg_alpha=0, reg_lambda=1.9, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

fold 2 round 2683 : score: 0.817544 | mean score 0.817943
auc:  0.7720995914167698



************************************************************************************************************************
Fold 3
************************************************************************************************************************
Training until validation scores don't improve for 500 rounds
[300]	valid_0's multi_logloss: 0.853347
[600]	valid_0's multi_logloss: 0.831441
[900]	valid_0's multi_logloss: 0.822029
[1200]	valid_0's multi_logloss: 0.817644
[1500]	valid_0's multi_logloss: 0.814483
[1800]	valid_0's multi_logloss: 0.812597
[2100]	valid_0's multi_logloss: 0.811762
[2400]	valid_0's multi_logloss: 0.811414
[2700]	valid_0's multi_logloss: 0.811051
[3000]	valid_0's multi_logloss: 0.81103
[3300]	valid_0's multi_logloss: 0.81074
[3600]	valid_0's multi_logloss: 0.811617
Early stopping, best iteration is:
[3285]	valid_0's multi_logloss: 0.810693


...

fold 3 round 3285 : score: 0.810693 | mean score 0.815526
auc:  0.7735997303916466



************************************************************************************************************************
Fold 4
************************************************************************************************************************
Training until validation scores don't improve for 500 rounds
[300]	valid_0's multi_logloss: 0.852726
[600]	valid_0's multi_logloss: 0.831451
[900]	valid_0's multi_logloss: 0.823418
[1200]	valid_0's multi_logloss: 0.818663
[1500]	valid_0's multi_logloss: 0.816263
[1800]	valid_0's multi_logloss: 0.814824
[2100]	valid_0's multi_logloss: 0.813922
[2400]	valid_0's multi_logloss: 0.813059
[2700]	valid_0's multi_logloss: 0.81256
[3000]	valid_0's multi_logloss: 0.81244
[3300]	valid_0's multi_logloss: 0.81276
Early stopping, best iteration is:
[2862]	valid_0's multi_logloss: 0.812334


LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.05, max_bin=64,
               max_depth=6, min_child_samples=80, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=10000, n_jobs=4, num_class=4,
               num_leaves=63, objective='multiclass', random_state=527,
               reg_alpha=0, reg_lambda=1.9, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

fold 4 round 2862 : score: 0.812334 | mean score 0.814728
auc:  0.7715348809312834



************************************************************************************************************************
Fold 5
************************************************************************************************************************
Training until validation scores don't improve for 500 rounds
[300]	valid_0's multi_logloss: 0.856857
[600]	valid_0's multi_logloss: 0.835541
[900]	valid_0's multi_logloss: 0.827005
[1200]	valid_0's multi_logloss: 0.822282
[1500]	valid_0's multi_logloss: 0.819842
[1800]	valid_0's multi_logloss: 0.818093
[2100]	valid_0's multi_logloss: 0.817241
[2400]	valid_0's multi_logloss: 0.817022
[2700]	valid_0's multi_logloss: 0.816802
[3000]	valid_0's multi_logloss: 0.817042
Early stopping, best iteration is:
[2643]	valid_0's multi_logloss: 0.816797


LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.05, max_bin=64,
               max_depth=6, min_child_samples=80, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=10000, n_jobs=4, num_class=4,
               num_leaves=63, objective='multiclass', random_state=527,
               reg_alpha=0, reg_lambda=1.9, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

fold 5 round 2643 : score: 0.816797 | mean score 0.815142
auc:  0.7719512792948682





In [13]:
#data_new fold1
# fold 1 round 2757 : score: 0.817910 | mean score 0.817910
# auc:  0.771650968641868

print(scores)
print('logloss:',np.mean(scores))
print('auc:', multi_auc_oof(train_y, oof_lgb))
print('acc:', accuracy_score(train_y.map(label_map), np.argmax(oof_lgb, axis=1)))
# logloss: 0.8394470890181213 auc: 0.7639691112376329 acc: 0.6468655840291672  featv0 0.75723 $
# logloss: 0.8386205747554791 auc: 0.7641546488063027 acc: 0.6471483155930496 featv1 0.75757 $
# logloss: 0.8385694282652787 auc: 0.7644872386334696 acc: 0.6476437690002342 527 527
# logloss: 0.83799263525929 auc: 0.7643224628185179 acc: 0.6474014276597635 featv2 527 527 0.75785 $
# logloss: 0.8230686055330182 auc: 0.7691376036021507 acc: 0.6545720386561364 featv3 0.76301 $
# logloss: 0.8153214789014147 auc: 0.772159018172669 acc: 0.6590661241810882 data_ 0.76654 $
# logloss: 0.8151705994830186 auc: 0.7722130382430792 acc: 0.6591495973094725 data_:1000 0.76649
# logloss: 0.8154789314371726 auc: 0.7720832773246235 acc: 0.6589530315555352 data_:1200 0.76588
# logloss: 0.8152194259449506 auc: 0.7722792166028103 acc: 0.6592411484825393 data_:800  0.76618
# logloss: 0.8157724809172372 auc: 0.7721843858406869 acc: 0.6591038217229392 data_:1400  0.76606
# logloss: 0.815500754326537 auc: 0.7718635245268304 acc: 0.6586245244051193 data_:1500  0.76559
# logloss: 0.8154257388311787 auc: 0.7719421140288143 acc: 0.6587456950753547 data_:1540 

# logloss: 0.8153689546673759 auc: 0.77196372533444 acc: 0.6587806999356449 data_new X 0.76571
# logloss: 0.8151418696703783 auc: 0.772302807868475 acc: 0.6592842313875119 data_new 0.76631

[0.8183417075797106, 0.8175437507146918, 0.8106925505809366, 0.8123341458862849, 0.816797193590267]
logloss: 0.8151418696703783
auc: 0.772302807868475
acc: 0.6592842313875119


In [14]:
cm = confusion_matrix(train_y.map(label_map), np.argmax(oof_lgb, axis=1))
acc0 = cm[0][0] / cm[0].sum()
acc1 = cm[1][1] / cm[1].sum()
acc2 = cm[2][2] / cm[2].sum()
acc3 = cm[3][3] / cm[3].sum()
cm
acc0, acc1, acc2, acc3

array([[63250, 13947,  4741, 10809],
       [23203, 44427, 14513, 10427],
       [ 7329, 14222, 59318, 10255],
       [ 7598,  3821,  5669, 77848]], dtype=int64)

(0.681962758903253,
 0.47992870260343523,
 0.6509591326105088,
 0.8200050560377518)

In [15]:
imp['score'] = imp['score1'] + imp['score2'] + imp['score3'] + imp['score4'] + imp['score5']
imp.sort_values(by='score', ascending=False)

,feat,score1,score2,score3,score4,score5,score
9,event_id_jet_px_sum,3152,3110,3638,3129,2989,16018
243,event_id_jet_id_particle_py_std_min,2902,2693,3127,2885,2790,14397
268,event_id_jet_id_particle_pz_std_min,2889,2720,3083,2938,2555,14185
6,event_id_jet_px_mean,2663,2533,3093,2904,2569,13762
145,event_id_particle_category_-211_std,2680,2462,2840,2630,2497,13109
763,event_id_jet_id_particle_energy_rel_sum_min,2579,2432,2915,2533,2442,12901
180,event_id_particle_category_211_std,2549,2380,2828,2491,2377,12625
783,event_id_jet_id_particle_R_min_min,2516,2285,2759,2391,2441,12392
791,event_id_jet_id_particle_R_std_mean,2341,2259,2607,2322,2111,11640
103,event_id_jet_eta_sinh_abs_min,2194,2153,2503,2152,2015,11017


In [16]:
# imp_array = np.array(imp.sort_values(by='score', ascending=False)['feat'])
# imp_array
# np.save('E:/jet_complex_data/imp.npy', imp_array)

In [17]:
test_jet = pd.read_csv('E:/jet_complex_data/complex_test_R04_jet.csv')

In [18]:
test['label'] = preds.argmax(axis=1)
test['label'] = test['label'].map(label_map_reverse).astype(np.int64)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
sub = test_jet.merge(test[['event_id', 'label']], on='event_id', how='left')[['jet_id', 'label']]
sub.rename(columns={'jet_id':'id'}, inplace = True)
sub

,id,label
0,cb3b5c6ea5e441e9b425fc1e6a4a00e6,5
1,838bf5c3f9bc4a2194f51b3c57f403cd,21
2,fbc245d8b90b4a5d804880e097244a67,1
3,f4c5dffa513a443fbbd27ca4c0fbe44f,21
4,74b105d4cb654c4d94da3fb88c8cd6a7,21
...,...,...
537944,e6d8625e7a804588bb5a03e97be8092e,21
537945,56f663b1a299408d905298bc1aef9ab3,4
537946,568150932e9f48cd86ccdb28faa29cff,4
537947,f3dc23154e874319a9d3775f8965975d,21


In [20]:
sub['label'].value_counts()
# 21    194142
# 1     135193
# 5     112430
# 4      96184

21    193798
1     135346
5     112029
4      96776
Name: label, dtype: int64

In [21]:
if not os.path.exists('E:/jet_complex_data/sub'):
    os.mkdir('E:/jet_complex_data/sub')
sub.to_csv('E:/jet_complex_data/sub/sub1-10_data_new.csv', index=False)

In [22]:
# imp_top = imp.sort_values(by='score', ascending=False)['feat'][:5].values
# imp_top = list(imp_top)
# imp_top.append('label')
# imp_top